## 감독선호도 구하기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
group10 = pd.read_csv(os.path.join('./group10.csv'))

In [ ]:
group10

,director,user,time,rank,num_mov,dir_loyalty
0,nm0000005,3201,10,73.0,19,52.631579
1,nm0000005,3284,12,46.0,19,63.157895
2,nm0000005,8405,17,9.0,19,89.473684
3,nm0000005,10098,10,74.0,19,52.631579
4,nm0000005,10286,11,54.0,19,57.894737
...,...,...,...,...,...,...
6501,nm0943758,119048,12,12.0,20,60.000000
6502,nm0943758,121870,15,4.0,20,75.000000
6503,nm0943758,127138,11,16.0,20,55.000000
6504,nm0943758,131904,11,17.0,20,55.000000


In [ ]:
(group10.groupby('director')['dir_loyalty'].min() >= 80).sum()

21

In [ ]:
temp= group10.groupby('director')['dir_loyalty'].min()
temp=pd.DataFrame(temp)
temp.reset_index(inplace=True)

In [ ]:
# 최소 로열티가 80% 이상인 21명의 감독들
dir_list= list(temp[pd.DataFrame(temp)['dir_loyalty'] >=80]['director'])

In [ ]:
# 21명의 감독 영화를 10개 이상 본 509명의 maniac 유저들
maniac= group10[group10['director'].isin(dir_list)][['director','user']]

In [ ]:
director_data = pd.read_csv(os.path.join('./data/train/directors.tsv'), sep='\t')

In [ ]:
# 21명 감독의 item id 뽑기
items= director_data[director_data['director'].isin(dir_list)]

In [ ]:
# 각 유저가 선호하는 감독의 영화 id 붙인 프레임 생성
pref_item= pd.merge(items,maniac,on='director',how='right')
pref_item.drop('director',axis=1, inplace=True)

# ID 변경
pref_item= numerize(pref_item, profile2id, show2id)

In [ ]:
# test용
tp_tr = pd.read_csv("./data/train/pro_sg/test_te.csv")
pref_test= pref_item[pref_item['uid'].isin(tp_tr['uid'])]

In [ ]:
# PREF ITEM inference용
n_items = loader.load_n_items()
n_users = 31360
rows, cols = pref_item['uid'], pref_item['sid']
pref_item= sparse.csr_matrix((np.ones_like(rows),
                                 (rows, cols)), dtype='float64',
                                 shape=(n_users, n_items))


In [ ]:
# test용

start_idx = tp_tr['uid'].min()
end_idx = tp_tr['uid'].max()

rows_tr, cols_tr = pref_test['uid'] - start_idx, pref_test['sid']

pref_test = sparse.csr_matrix((np.ones_like(rows_tr),
                            (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, 6807))

In [ ]:
print(pref_item.toarray()[19629][1038])

1.0


In [ ]:
# 1인 곳에 가중치 1.8
pref_item0= pref_item.copy()
pref_test0= pref_test.copy()

pref_item0[pref_item0==1.0] *= 2
pref_test0[pref_test0==1.0] *= 2

In [ ]:
#0은 1로
pref_item0[pref_item0==0] =1
pref_test0[pref_test0==0] =1

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
pref_item0.toarray()[19629][1038]

2.0

In [ ]:
# array로 변경
pref_item0= pref_item0.toarray()
pref_test0= pref_test0.toarray()

In [ ]:
with open('pref_item0.npy', 'wb')as f:
    np.save(f, pref_item0)
with open('pref_test0.npy', 'wb')as f:
    np.save(f, pref_test0)